In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--lang=es')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
dict_university = {
    'Trường Đại học Bách khoa Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+B%C3%A1ch+khoa+H%C3%A0+N%E1%BB%99i/@21.0056028,105.8347192,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac76ccab6dd7:0x55e92a5b07a97d03!8m2!3d21.005603!4d105.8434525!9m1!1b1',
    'Trường Đại học Thăng Long': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Th%C4%83ng+Long/@20.9758757,105.8068602,15z/data=!4m10!1m2!2m1!1suniversity!3m6!1s0x3135acef8ad5350f:0x89435a3528118ff5!8m2!3d20.9758759!4d105.8155935!9m1!1b1',
    'Trường Đại học Ngoại ngữ': 'https://www.google.it/maps/place/ULIS/@21.0392422,105.7730808,15z/data=!4m10!1m2!2m1!1suniversity!3m6!1s0x313454cacd93cc57:0x465a0246eecc8c72!8m2!3d21.0392424!4d105.7818141!9m1!1b1',
    'Trường Đại học Giao thông Vận tải': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Giao+th%C3%B4ng+V%E1%BA%ADn+t%E1%BA%A3i+(UTC)/@21.0281543,105.7946872,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ab424a50fff9:0xbe3a7f3670c0a45f!8m2!3d21.0281545!4d105.8034205!9m1!1b1',
    'Trường Đại Học Kinh Tế Quốc Dân': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+H%E1%BB%8Dc+Kinh+T%E1%BA%BF+Qu%E1%BB%91c+D%C3%A2n/@20.9998293,105.8334548,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac71752d8f79:0xd2ec575c01017afa!8m2!3d20.9998295!4d105.8421881!9m1!1b1',
    'Trường Đại Học Thủy Lợi': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+H%E1%BB%8Dc+Th%E1%BB%A7y+L%E1%BB%A3i/@21.0073826,105.8159904,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac8109765ba5:0xd84740ece05680ee!8m2!3d21.0073828!4d105.8247237!9m1!1b1',
    'Trường Đại học Thương mại': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Th%C6%B0%C6%A1ng+m%E1%BA%A1i/@21.0367007,105.7662902,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x313454b6163c392f:0x1ebf64facbb56d03!8m2!3d21.0367009!4d105.7750235!9m1!1b1',
    'Trường Đại học Khoa học Xã hội và Nhân văn - ĐHQGHN': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Khoa+h%E1%BB%8Dc+X%C3%A3+h%E1%BB%99i+v%C3%A0+Nh%C3%A2n+v%C4%83n/@20.9951643,105.7984947,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac9642e7777d:0x412403da0c4dd792!8m2!3d20.9951645!4d105.807228!9m1!1b1',
    'Trường Đại Học Xây Dựng': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+H%E1%BB%8Dc+X%C3%A2y+D%E1%BB%B1ng/@21.0033133,105.8345874,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac773026b415:0x499b8b613889f78a!8m2!3d21.0033135!4d105.8433207!9m1!1b1',
    'Trường Đại học Ngoại thương Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Ngo%E1%BA%A1i+th%C6%B0%C6%A1ng+H%C3%A0+N%E1%BB%99i/@21.0230328,105.7967037,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ab5d5161f909:0x4dafaf500ce22be3!8m2!3d21.023033!4d105.805437!9m1!1b1',
    'Trường Đại học Luật Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Lu%E1%BA%ADt+H%C3%A0+N%E1%BB%99i/@21.0214655,105.8009416,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ab666ac2c3b1:0x8f77b30b5f366910!8m2!3d21.0214657!4d105.8096749!9m1!1b1',
    'Trường Đại học Dược Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+D%C6%B0%E1%BB%A3c+H%C3%A0+N%E1%BB%99i/@21.021521,105.8494321,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135abee82135aa5:0x42bedb896d6f3ce9!8m2!3d21.0215212!4d105.8581654!9m1!1b1',
    'Trường Đại Học Y Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+H%E1%BB%8Dc+Y+H%C3%A0+N%E1%BB%99i/@21.0032312,105.8219591,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac7c33fd6323:0x930d205cf25ff492!8m2!3d21.0032314!4d105.8306924!9m1!1b1',
    'Trường Đại học Khoa học Tự nhiên - ĐHQGHN': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Khoa+h%E1%BB%8Dc+T%E1%BB%B1+nhi%C3%AAn,+%C4%90HQGHN/@20.995872,105.7992439,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135acbf0df2c0e5:0xd740a66998e1a0ed!8m2!3d20.9958722!4d105.8079772!9m1!1b1',
    'Trường Đại Học Kiến Trúc Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+H%E1%BB%8Dc+Ki%E1%BA%BFn+Tr%C3%BAc+H%C3%A0+N%E1%BB%99i/@20.9805522,105.7806332,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135acce43a64245:0xdb176021a520454d!8m2!3d20.9805524!4d105.7893665!9m1!1b1',
    'Trường Đại học Sư phạm Hà Nội': 'https://www.google.it/maps/place/HNUE/@21.0374708,105.7746295,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ab355cc2239b:0x9ae247114fb38da3!8m2!3d21.037471!4d105.7833628!9m1!1b1',
    'Trường Đại học Kinh doanh và Công nghệ Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Kinh+doanh+v%C3%A0+C%C3%B4ng+ngh%E1%BB%87+H%C3%A0+N%E1%BB%99i/@20.9996914,105.8692895,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135aeaa17c35b81:0x79d8becf2f06f8dc!8m2!3d20.9996916!4d105.8780228!9m1!1b1',
    'Trường Đại học Mỹ thuật Công nghiệp': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+M%E1%BB%B9+thu%E1%BA%ADt+C%C3%B4ng+nghi%E1%BB%87p/@21.0235933,105.8149384,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ab79d9e00c97:0xb18bdf08f7889a6!8m2!3d21.0235735!4d105.8236717!9m1!1b1',
    'Trường Đại học Công nghiệp Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+C%C3%B4ng+nghi%E1%BB%87p+H%C3%A0+N%E1%BB%99i/@21.0544692,105.7264087,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x31345457e292d5bf:0x20ac91c94d74439a!8m2!3d21.0544494!4d105.735142!9m1!1b1',
    'Trường Đại học Sân khấu Điện ảnh Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+S%C3%A2n+kh%E1%BA%A5u+%C4%90i%E1%BB%87n+%E1%BA%A3nh+H%C3%A0+N%E1%BB%99i/@21.0400206,105.7671644,15z/data=!4m10!1m2!2m1!1suniversity!3m6!1s0x313454c99118ed37:0x47ae4d512aa9200b!8m2!3d21.0400008!4d105.7758977!9m1!1b1',
    'Trường Đại học Hà Nội': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+H%C3%A0+N%E1%BB%99i/@20.9892627,105.7866204,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135adb29ed54487:0xbe22035eae87b5d7!8m2!3d20.9892429!4d105.7953537!9m1!1b1',
    'Học viện Báo chí & Tuyên truyền': 'https://www.google.it/maps/place/H%E1%BB%8Dc+vi%E1%BB%87n+B%C3%A1o+ch%C3%AD+%26+Tuy%C3%AAn+truy%E1%BB%81n/@21.037143,105.7801806,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ab3613967dbf:0x3e1ad0c809d813b7!8m2!3d21.0371232!4d105.7889139!9m1!1b1',
    'Học viện Ngân hàng': 'https://www.google.it/maps/place/H%E1%BB%8Dc+vi%E1%BB%87n+Ng%C3%A2n+h%C3%A0ng/@21.008772,105.8198572,15z/data=!3m1!4b1!4m10!1m2!2m1!1suniversity!3m6!1s0x3135ac800f450807:0x419a49bcd94b693a!8m2!3d21.0087522!4d105.8285905!9m1!1b1',
    'Trường Đại học Công nghệ - ĐHQGHN': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+C%C3%B4ng+ngh%E1%BB%87,+%C4%90%E1%BA%A1i+h%E1%BB%8Dc+Qu%E1%BB%91c+gia+H%C3%A0+N%E1%BB%99i/@21.0382573,105.7739743,15z/data=!3m1!4b1!4m10!1m2!2m1!1zxJFhzKNpIGhvzKNj!3m6!1s0x3135ab354920c233:0x5d0313a3bfdc4f37!8m2!3d21.0382375!4d105.7827076!9m1!1b1',
    'Trường Đại Học FPT': 'https://www.google.it/maps/place/Tr%C6%B0%E1%BB%9Dng+%C4%90%E1%BA%A1i+H%E1%BB%8Dc+FPT/@21.0132698,105.518331,15z/data=!3m1!4b1!4m7!3m6!1s0x31345b465a4e65fb:0xaae6040cfabe8fe!8m2!3d21.01325!4d105.5270643!9m1!1b1',
    'Học viện Ngoại giao': 'https://www.google.it/maps/place/H%E1%BB%8Dc+vi%E1%BB%87n+Ngo%E1%BA%A1i+giao/@21.0229891,105.8043203,17z/data=!4m10!1m2!2m1!1zaG_Mo2MgdmnDqsyjbg!3m6!1s0x3135ab662a9c636d:0x24f4e1bf14505b3c!8m2!3d21.0229841!4d105.806509!9m1!1b1',
    'Học Viện Kỹ Thuật Mật Mã': 'https://www.google.it/maps/place/H%E1%BB%8Dc+Vi%E1%BB%87n+K%E1%BB%B9+Thu%E1%BA%ADt+M%E1%BA%ADt+M%C3%A3/@20.9808164,105.7940398,17z/data=!3m1!4b1!4m10!1m2!2m1!1zaG_Mo2MgdmnDqsyjbg!3m6!1s0x3135acc508f938fd:0x883e474806a2d1f2!8m2!3d20.9808114!4d105.7962285!9m1!1b1',
    'Học Viện An ninh Nhân Dân': 'https://www.google.it/maps/place/H%E1%BB%8Dc+Vi%E1%BB%87n+An+ninh+Nh%C3%A2n+D%C3%A2n/@20.981876,105.7893306,17z/data=!3m1!4b1!4m10!1m2!2m1!1zaG_Mo2MgdmnDqsyjbg!3m6!1s0x3135accf04011535:0xa93dfa61fb043aaf!8m2!3d20.981871!4d105.7915193!9m1!1b1',
    'Học viện Báo chí & Tuyên truyền': 'https://www.google.it/maps/place/H%E1%BB%8Dc+vi%E1%BB%87n+B%C3%A1o+ch%C3%AD+%26+Tuy%C3%AAn+truy%E1%BB%81n/@21.0371282,105.7867252,17z/data=!3m1!4b1!4m10!1m2!2m1!1zaG_Mo2MgdmnDqsyjbg!3m6!1s0x3135ab3613967dbf:0x3e1ad0c809d813b7!8m2!3d21.0371232!4d105.7889139!9m1!1b1',
    'Học Viện Âm Nhạc Quốc Gia Việt Nam': 'https://www.google.it/maps/place/H%E1%BB%8Dc+Vi%E1%BB%87n+%C3%82m+Nh%E1%BA%A1c+Qu%E1%BB%91c+Gia+Vi%E1%BB%87t+Nam/@21.0231236,105.8238819,17z/data=!3m1!4b1!4m10!1m2!2m1!1zaG_Mo2MgdmnDqsyjbg!3m6!1s0x3135ab77a0080205:0x5b451235586cb204!8m2!3d21.0231186!4d105.8260706!9m1!1b1',
    'Học Viện Tài Chính': 'https://www.google.it/maps/place/H%E1%BB%8Dc+Vi%E1%BB%87n+T%C3%A0i+Ch%C3%ADnh/@21.0754199,105.7705796,17z/data=!4m7!3m6!1s0x3134553054bc3a9b:0x34a690fa7ab05d1e!8m2!3d21.0754149!4d105.7727683!9m1!1b1',
    'Học viện Quân y': 'https://www.google.it/maps/place/H%E1%BB%8Dc+vi%E1%BB%87n+Qu%C3%A2n+y/@20.9665631,105.7862487,17z/data=!4m7!3m6!1s0x3135ad29ceae9933:0x2975955fde1ef478!8m2!3d20.9665581!4d105.7884374!9m1!1b1',
    'Học Viện Cảnh Sát Nhân Dân': 'https://www.google.it/maps/place/H%E1%BB%8Dc+Vi%E1%BB%87n+C%E1%BA%A3nh+S%C3%A1t+Nh%C3%A2n+D%C3%A2n+-+T32/@21.0662103,105.7618714,17z/data=!4m10!1m2!2m1!1zaOG7jWMgdmnhu4duIGPhuqNuaCBzw6F0!3m6!1s0x3134552147a20d85:0x44b22da812f8a618!8m2!3d21.0662053!4d105.7640601!9m1!1b1'
}

In [ ]:
list_university = list(dict_university.keys())

In [ ]:
university = pd.DataFrame(columns = ['University_name', 'University_rating', 'Review', 'Rating', 'Time'])

for uni in list_university:
    url = dict_university[uni] + '?hl=vi'
    driver.get(url)
    time.sleep(5)

    for i in range(200):
        scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(0.4)

    for i in range(2):
        for button in driver.find_elements_by_xpath('//button[@class="section-expand-review blue-link"]'):
            button.click()
    #except:
    #   print('There is error about scroll or expand button in {}'.format(uni))
    #   continue

    bs = BeautifulSoup(driver.page_source, 'html.parser')

    university_rating = bs.find('div', class_='gm2-display-2').text
    review_block = bs.find_all('span', class_='section-review-text')
    star_block = bs.find_all('span', class_='section-review-stars')
    time_block = bs.find_all('span', class_='section-review-publish-date')

    for i in range(len(review_block)):
        university.loc[len(university)] = [uni, university_rating, review_block[i].text, star_block[i]['aria-label'], time_block[i].text]

In [ ]:
university.to_csv('university.csv', index=False)